In [ ]:
!pip install scikit-neuralnetwork 

In [ ]:
!pip install git+https://github.com/scikit-learn/scikit-learn.git

In [ ]:
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_63f2dc1df232458db8431e5111d801de(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_5522e9053c58b1890eaff6e6cff76825d6927ffe','domain': {'id': 'bff414f8f6fe40c7a77ed050397d07db'},
            'password': 'GWb0*z/6v3/ln!l('}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

print("Start downloading quarter data")
df_data_3 = pd.read_csv(get_object_storage_file_with_credentials_63f2dc1df232458db8431e5111d801de('DefaultProjectmahajanvhuskyneuedu', '2005Q1.csv'))


df_data_4 = pd.read_csv(get_object_storage_file_with_credentials_63f2dc1df232458db8431e5111d801de('DefaultProjectmahajanvhuskyneuedu', '2005Q2.csv'))


dataframe=df_data_3
dataframe_quarter=df_data_4

#print(dataframe["ORIGINAL_INTEREST_RATE"])
print("Data farmes for both quarter's created")
#or download from cache
#dataframe=pd.read_csv("2005Q1.csv")
#dataframe_quarter = pd.read_csv("2005Q2.csv")


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.cross_validation import train_test_split, KFold, StratifiedShuffleSplit,StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model

def handle_missing_values(df):
    
    # all col_names that require "unknown" to be the replacement value
    Acquisition_names_unknown = [
                     "MORTAGAGE_INSURANCE_PERCENTAGE",
                     "NUMBER_OF_UNITS",
                     "OCCUPANCY_STATUS",
#                      "ORIGINAL_DEBT_TO_INCOME_RATIO", # to-do "   " i.e 3 space with 65, null with "Unknown"
                     "ORIGINAL_UPB",
                     "ORIGINAL_LOAN_TO_VALUE",
                     
                     "CHANNEL",
                     "PREPAYMENT_PENALTY_MORTGAGE_FLAG",
#                      "PRODUCT_TYPE",
                     "PROPERTY_STATE",
                     "PROPERTY_TYPE",
                     "POSTAL_CODE",
#                      "LOAN_SEQUENCE_NUMBER",
                     "LOAN_PURPOSE",
#                      "ORIGINAL_LOAN_TERM",
                     "NUMBER_OF_BORROWERS",
                     "SELLER_NAME",
                     "SERVICER_NAME",
#                      "SUPER_CONFORMING_FLAG"
                    ]
    
    for col_name in Acquisition_names_unknown:
        new_val = "Unknown"
        df[col_name]= np.where(pd.isnull(df[col_name]) ,new_val,df[col_name] )
    print("Replacing null values with 'Unknown' in certain columns complete!")
    
#     CREDIT_SCORE
    new_val = 300
    df['CREDIT_SCORE']= np.where(pd.isnull(df['CREDIT_SCORE']) ,new_val,df['CREDIT_SCORE'] )    
        
            
    #FIRST_PAYMENT_DATE
#   df['FIRST_PAYMENT_DATE'] = pd.to_datetime(df['FIRST_PAYMENT_DATE'], format='%Y%m')
#    print("FIRST_PAYMENT_DATE conversion complete!")
    
    
    #FIRST_TIME_HOMEBUYER_FLAG
    #Replace space with “Not Applicable”
    new_val = "Not Applicable"
    df['FIRST_TIME_HOMEBUYER_FLAG']= np.where(pd.isnull(df['FIRST_TIME_HOMEBUYER_FLAG']) ,new_val,df['FIRST_TIME_HOMEBUYER_FLAG'] )

    #MATURITY_DATE
    #df['MATURITY_DATE'] = pd.to_datetime(df['MATURITY_DATE'], format='%Y%m')
    #print("MATURITY_DATE conversion complete!")
    #Replace missing values with : check original loan term/12 to get the number of year and add it to the First payment due date

    #MSA
    #Replace missing values: “Neither MSA nor MD/Unknown”
    new_val = "Neither MSA nor MD/Unknown"
    df['MSA']= np.where(pd.isnull(df['MSA']) ,new_val,df['MSA'] )


     #ORGINAL_COMBINED_LOAN_TO_VALUE Look into this (imp feature)
    #Replace missing values: “Unknown”
    
    new_val = "Unknown"
    df['ORGINAL_COMBINED_LOAN_TO_VALUE']= np.where(pd.isnull(df['ORGINAL_COMBINED_LOAN_TO_VALUE']) ,new_val,df['ORGINAL_COMBINED_LOAN_TO_VALUE'] )
    
    
    #ORIGINAL_DEBT_TO_INCOME_RATIO
    #Replace "Spaces in ORIGINAL_DEBT_TO_INCOME_RATIO replaced with > 65"
    new_val = 66
    df['ORIGINAL_DEBT_TO_INCOME_RATIO']= np.where(pd.isnull(df['ORIGINAL_DEBT_TO_INCOME_RATIO']) ,new_val,df['ORIGINAL_DEBT_TO_INCOME_RATIO'] )
    
    
    #ORIGINAL_INTEREST_RATE
    #Replace missing values: mean
    df['ORIGINAL_INTEREST_RATE'] = df['ORIGINAL_INTEREST_RATE'].astype('float')

    new_val =  df['ORIGINAL_INTEREST_RATE'].mean()

    df['ORIGINAL_INTEREST_RATE']= np.where(pd.isnull(df['ORIGINAL_INTEREST_RATE']) ,new_val,df['ORIGINAL_INTEREST_RATE'] )

    print("Replacing null values with 'mean' in transformed_df complete!")
    
    #PRODUCT_TYPE
    #Replace missing values: “No”
    new_val = "FRM"
    df['PRODUCT_TYPE']= np.where(pd.isnull(df['PRODUCT_TYPE']) ,new_val,df['PRODUCT_TYPE'] )


     #SUPER_CONFORMING_FLAG
    #Replace missing values: “No”
    new_val = "No"
    df['SUPER_CONFORMING_FLAG']= np.where(pd.isnull(df['SUPER_CONFORMING_FLAG']) ,new_val,df['SUPER_CONFORMING_FLAG'] )

    print("Handling missing /null data in other columns!")
       
    return df
    

def analysis(res_new):
    
    #create a new column credit bucket
    res_new.insert(1, "CREDIT_BUCKET", "N/A")
    
    #categorize the user in medium/good credit standing on the basis of their credit score (threshold set as 650)
    count=0
    for i in res_new:
        if(int(res_new["CREDIT_SCORE"][count]) <=650):
            res_new["CREDIT_BUCKET"][count]="Medium Credit"
        else:
            res_new["CREDIT_BUCKET"][count]="Good Credit"
        count=count+1
    
    
    #calculate the gross income using the original_debt_to_income and unpaid_principal_balance using formula (UPB * 100/DI)
    #Assumption: that the user has only one property in which he has done investment
    res_new.insert(8, "GROSS_INCOME", 0)
    res_new["GROSS_INCOME"]=res_new["GROSS_INCOME"]
    print("Calculating gross income")
    count_for_gross_income=0
    error_count=0

    for i in res_new:
        if( res_new["ORIGINAL_DEBT_TO_INCOME_RATIO"][count_for_gross_income] == "   "):
            print("ORIGINAL_DEBT_TO_INCOME_RATIO null at ",count_for_gross_income," value>>",res_new["ORIGINAL_DEBT_TO_INCOME_RATIO"][count_for_gross_income],"<<")
        try:
            
            res_new["GROSS_INCOME"][count_for_gross_income]=int(res_new["ORIGINAL_UPB"][count_for_gross_income]) * 100 / (int(res_new["ORIGINAL_DEBT_TO_INCOME_RATIO"][count_for_gross_income]))
            count_for_gross_income=count_for_gross_income+1
        except Exception as e:
            if(error_count == 1):
                
                print(" ORIGINAL_UPB >>>>>>>>>>>>",res_new["ORIGINAL_UPB"][count_for_gross_income],"<<<<")
                print(" ORIGINAL_DEBT_TO_INCOME_RATIO>>>>>>>>>", res_new["ORIGINAL_DEBT_TO_INCOME_RATIO"][count_for_gross_income],"<<<<<",len(res_new["ORIGINAL_DEBT_TO_INCOME_RATIO"][count_for_gross_income]))
               
                error_count+=1
        
    print("Gross income computed and added to the dataframe")
   
    #calculate the gross_income_less_fixed_monthly_expenses using GROSS_INCOME - unpaid_principal_balance - additional expenses
    # Note: Note: As per Bureau of Statistics the average expenditure of a married couple with children is 13% of their gross income
    # adding an extra column, "gross_income_less_fixed_monthly_expenses"
    res_new.insert(9, "GROSS_INCOME_LESS_FIXED_MONTHLY_EXPENSES", 0)
    
    cnt=0
    for i in res_new:
        res_new["GROSS_INCOME_LESS_FIXED_MONTHLY_EXPENSES"][cnt]=int(res_new["GROSS_INCOME"][cnt])- int(res_new["ORIGINAL_UPB"][cnt])- (int(res_new["GROSS_INCOME"][cnt])*0.13)
        cnt=cnt+1
        
    print("Gross income less fixed monthly expenses computed and added the dataframe")
    
    #compute the "Mortgage Qualification Amount" as per the formula mentioned below:
    # if user is in "Medium Credit" standing take minimum of (28% of gross income,36% of gross income less fixed monthly expenses)
    # else if user is in "Good Credit" standing take minimum of (36% of gross income,42% of gross income less fixed monthly expenses)
    #Refered these figures from http://tcalc.timevalue.com/all-financial-calculators/mortgage-calculators/mortgage-qualification-calculator.aspx
    res_new.insert(10, "MORTGAGE_QUALIFICATION_AMOUNT", 0)
    res_new.insert(11, "DEVIATION_IN_SANTIONED_AMOUNT", 0)
    
    print("exporting clean and merged dataframe!!!")
    
    small_df=res_new.head(5)
    count_for_mortgage_qualificatin_amt=0
    for i in res_new["CREDIT_BUCKET"]:
        if(i=="Medium Credit"):
            if((int(res_new["GROSS_INCOME"][count_for_mortgage_qualificatin_amt])*0.28) < (int(res_new["GROSS_INCOME_LESS_FIXED_MONTHLY_EXPENSES"][count_for_mortgage_qualificatin_amt])*0.36)):
                res_new["MORTGAGE_QUALIFICATION_AMOUNT"][count_for_mortgage_qualificatin_amt]=(int(res_new["GROSS_INCOME"][count_for_mortgage_qualificatin_amt])*0.28)
               
            else:
                res_new["MORTGAGE_QUALIFICATION_AMOUNT"][count_for_mortgage_qualificatin_amt]=(int(res_new["GROSS_INCOME_LESS_FIXED_MONTHLY_EXPENSES"][count_for_mortgage_qualificatin_amt])*0.36)
                
        else:
            if((int(res_new["GROSS_INCOME"][count_for_mortgage_qualificatin_amt])*0.36) < (int(res_new["GROSS_INCOME_LESS_FIXED_MONTHLY_EXPENSES"][count_for_mortgage_qualificatin_amt])*0.42)):
                res_new["MORTGAGE_QUALIFICATION_AMOUNT"][count_for_mortgage_qualificatin_amt]=(int(res_new["GROSS_INCOME"][count_for_mortgage_qualificatin_amt])*0.36)
               
            else:
                res_new["MORTGAGE_QUALIFICATION_AMOUNT"][count_for_mortgage_qualificatin_amt]=(int(res_new["GROSS_INCOME_LESS_FIXED_MONTHLY_EXPENSES"][count_for_mortgage_qualificatin_amt])*0.42)
               
        count_for_mortgage_qualificatin_amt=count_for_mortgage_qualificatin_amt+1
    
    print("Mortgage Qualification Amount computed and added to the dataframe")
    
    
    #compute percentage difference between santioned loan amount and computed mortgage qualification amount
   
    count_for_deviation=0
    for i in res_new["DEVIATION_IN_SANTIONED_AMOUNT"]:
        res_new["DEVIATION_IN_SANTIONED_AMOUNT"][count_for_deviation]=((int(res_new["ORIGINAL_UPB"][count_for_deviation])-int(res_new["MORTGAGE_QUALIFICATION_AMOUNT"][count_for_deviation]))/int(res_new["ORIGINAL_UPB"][count_for_deviation]))*100
        count_for_deviation=count_for_deviation+1
        
    print("Percentage deviation between santioned and qualified loan amount computed")
    
    return df

#using label encoder to convert categorical columns into numeric values
def dummyEncode(df):
        columnsToEncode = list(df.select_dtypes(include=['category','object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                if feature!='ORIGINAL_INTEREST_RATE':
                    df[feature] = le.fit_transform(df[feature])
                else:
                    df[feature] = df[feature] 
            except:
                print('Error encoding '+feature)
        return df
    
print("Processing data columns started")
dataframe['PREPAYMENT_PENALTY_MORTGAGE_FLAG'].astype(bool)
dataframe['ORIGINAL_DEBT_TO_INCOME_RATIO']=dataframe['ORIGINAL_DEBT_TO_INCOME_RATIO'].convert_objects(convert_numeric=True)
dataframe['MORTAGAGE_INSURANCE_PERCENTAGE']=dataframe['MORTAGAGE_INSURANCE_PERCENTAGE'].convert_objects(convert_numeric=True)
dataframe['FIRST_TIME_HOMEBUYER_FLAG'].astype(bool)

dataframe = dataframe.fillna(method='ffill')
#dataframe=handle_missing_values(dataframe)
#dataframe=analysis(dataframe)
print(dataframe.head(2))

#print(dataframe.isnull().any())


dataframe_quarter['PREPAYMENT_PENALTY_MORTGAGE_FLAG'].astype(bool)
dataframe_quarter['FIRST_TIME_HOMEBUYER_FLAG'].astype(bool)
dataframe_quarter['ORIGINAL_DEBT_TO_INCOME_RATIO']=dataframe_quarter['ORIGINAL_DEBT_TO_INCOME_RATIO'].convert_objects(convert_numeric=True)
dataframe_quarter['MORTAGAGE_INSURANCE_PERCENTAGE']=dataframe_quarter['MORTAGAGE_INSURANCE_PERCENTAGE'].convert_objects(convert_numeric=True)
dataframe = dataframe.fillna(method='ffill')
#dataframe_quarter=handle_missing_values(dataframe_quarter)
#dataframe_quarter=analysis(dataframe_quarter)
print(dataframe_quarter.head(2))
#dataframe_quarter = dataframe_quarter.fillna(method='ffill')

print("Processing data columns completed")
#dataframe_quarter['ORIGINAL_DEBT_TO_INCOME_RATIO'].astype(float)
#feature selection based on coefficient values
print("New data frame with feature columns extracted")
feature_cols_dataframe=dataframe[[ "MATURITY_DATE",
                                    "ORIGINAL_UPB",
                                    "OCCUPANCY_STATUS",
                                    "CREDIT_SCORE",
                                    "PROPERTY_STATE",
                                    "ORGINAL_COMBINED_LOAN_TO_VALUE",
                                    "CHANNEL", 
                                    "SELLER_NAME",
                                    "PROPERTY_TYPE",
                                    "LOAN_PURPOSE",
                                    "ORIGINAL_INTEREST_RATE"
                                ]]
feature_cols_dataframe_quarter=dataframe_quarter[[ "MATURITY_DATE",
                                    "ORIGINAL_UPB",
                                    "OCCUPANCY_STATUS",
                                    "CREDIT_SCORE",
                                    "PROPERTY_STATE",
                                    "ORGINAL_COMBINED_LOAN_TO_VALUE",
                                    "CHANNEL", 
                                    "SELLER_NAME",
                                    "PROPERTY_TYPE",
                                    "LOAN_PURPOSE",
                                    "ORIGINAL_INTEREST_RATE"
                                ]]


#feature_cols=["PROPERTY_STATE",'PROPERTY_TYPE',"LOAN_PURPOSE",'SERVICER_NAME','ORIGINAL_INTEREST_RATE']

#transforming categorical/string data into numeric data for the algorithm
#used custom encoding instead of one hot encoding for memory efficiency
print("Transforming categorical values into numerical representation")
transformed_df=dummyEncode(feature_cols_dataframe)

transformed_df_next_quarter=dummyEncode(feature_cols_dataframe_quarter)

print("Creating test and training data")


Y_train=transformed_df.ORIGINAL_INTEREST_RATE
#print(Y_train["ORIGINAL_INTEREST_RATE"])
#transformed_df.drop('ORIGINAL_INTEREST_RATE')

X_train=transformed_df
#transformed_df['ORIGINAL_INTEREST_RATE']=transformed_df['ORIGINAL_INTEREST_RATE'].astype(float)


Y_test=transformed_df_next_quarter.ORIGINAL_INTEREST_RATE
#transformed_df_next_quarter.drop('ORIGINAL_INTEREST_RATE')
X_test=transformed_df_next_quarter
#transformed_df_next_quarter['ORIGINAL_INTEREST_RATE']=transformed_df_next_quarter['ORIGINAL_INTEREST_RATE'].astype(float)

print("Start running models on your data")
model_number = input("Choose the model that you want to run:"
                      "  A. Linear Regression "
                      "  B. Neural Network "
                      "  C. KNN"
                      "  D. Random Forest")

if model_number =='A':
    
    linear_reg = linear_model.SGDRegressor(loss='epsilon_insensitive',penalty='elasticnet')
    linear_reg.fit(X_train, Y_train)
    print ("Intercept is ",linear_reg.intercept_)
    print("Coefficient is ",linear_reg.coef_)
    #print(lm.predict([18,3,0,4]))
    #print("Training score is ",linear_reg.score(X_train, Y_train))
    y_pred=linear_reg.predict(X_test)
    #print("Testing score is ",linear_reg.score(X_test, Y_test))

    print (" Linear regression Mean Absolute Error is ",mean_absolute_error(Y_train,y_pred)*100)
    #print("Linear regression Mean Absolute Percentage Error is",mean_absolute_percentage_error(Y_train,y_pred))
    print("Linear regression Root Mean Squared Error",np.sqrt(mean_squared_error(Y_train,y_pred)))
    print("Linear regression R2 Score of the model is ",r2_score(Y_train,y_pred))
    

elif model_number =='B':
        
        #neural network
    scaler = StandardScaler()
    # Fit only to the training data
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    neural_network_reg=MLPRegressor(hidden_layer_sizes=(20,10,20))
    neural_network_reg.fit(X_train,Y_train)
    predictions = neural_network_reg.predict(X_test)
    y_pred=predictions
    #print("Intercept is ",neural_network_reg.intercept_)
    #print("Coefficient is ",neural_network_reg.coef_)
    # print("Training score is ",neural_network_reg.score(X_train, Y_train))
    # print("Testing score is ",neural_network_reg.score(X_test, Y_test))
    print ("Neural Network Mean Absolute Error is ",mean_absolute_error(Y_train,y_pred)*100)
    #print("Neural Network Mean Absolute Percentage Error is",mean_absolute_percentage_error(Y_train,y_pred))
    print("Neural Network Root Mean Squared Error",np.sqrt(mean_squared_error(Y_train,y_pred)))
    print("Neural Network R2 Score of the model is ",r2_score(Y_train,y_pred))

elif model_number == "C":
        #KNN algorithm
        print("Starting KNN algorithm")
        for K in range(25):
                 K_value = K+1
                 knn_reg = KNeighborsRegressor(n_neighbors = K_value, weights='uniform', algorithm='auto')
                 knn_reg.fit(X_train, Y_train)
                 y_pred = knn_reg.predict(X_test)
                 print("Mean Absolute Error is ",mean_absolute_error(Y_train,y_pred),"% for K-Value:",K_value)
                 #print("Mean Absolute Percentage Error is ",mean_absolute_percentage_error(Y_train,y_pred))
                 print("Root Mean Squared Error ",np.sqrt(mean_squared_error(Y_train,y_pred)))
                 print("Accuracy of the model is ",r2_score(Y_train,y_pred))

elif model_number == "D":
        print("Staring Random forest algorithm")
        random_forest = RandomForestRegressor(n_jobs=2)
        random_forest.fit(X_train, Y_train)
        y_pred=random_forest.predict(X_test)
        #predict probability of first 10 records
        #print(random_forest.predict_proba(X_test)[0:10])
        print("Mean Absolute Error is ",mean_absolute_error(Y_train,y_pred))
        #print("Mean Absolute Percentage Error is ",mean_absolute_percentage_error(Y_train,y_pred))
        print("Root Mean Squared Error ",np.sqrt(mean_squared_error(Y_train,y_pred)))
        print("Accuracy of the model is ",r2_score(Y_train,y_pred))     
        
else:
        print("Running all four algorithms in parallel")
        

        